In [ ]:
import logging
import os

import joblib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import seaborn as sns
from catboost import CatBoostRegressor
from rdkit import Chem, DataStructs
from rdkit.Chem import  AllChem
from rdkit.Chem.Fingerprints import FingerprintMols
from sklearn.metrics import mean_absolute_error as mae
from sklearn.metrics import r2_score
from sklearn.model_selection import KFold, train_test_split
from sklearn.svm import LinearSVR
from torch.utils.data import DataLoader, Dataset
from torchinfo import summary
from tqdm.contrib import tmap
from tqdm.contrib.concurrent import process_map, thread_map
from tqdm.notebook import tqdm
from mordred import Calculator

from rdkit.Chem.rdMolDescriptors import CalcMolFormula

# np.set_printoptions(threshold=sys.maxsize)
%matplotlib inline


In [ ]:
nist17_pred = pd.read_csv("../Data/preds_nist.csv")


In [ ]:
cols = ["RI_1D", "RI_MLP", "RI_CB", "RI_2D", "RI_SVR"]
nist17_pred["p_std"] = nist17_pred.loc[:, cols].std(axis=1)
nist17_pred["p_mean"] = nist17_pred.loc[:, cols].mean(axis=1)
nist17_pred["p_median"] = nist17_pred.loc[:, cols].median(axis=1)
nist17_pred["d_median"] = (nist17_pred.loc[:, "RI_X"] -
                       nist17_pred.loc[:, "p_median"]).abs()
nist17_pred["d_median_R"] = (nist17_pred["d_median"] /
                         nist17_pred.loc[:, "p_median"]).abs()


In [ ]:
for i in [5, 10, 15, 25]:
    nist17_pred[f"S_{i}"] = np.zeros(len(nist17_pred))
    nist17_pred[f"S_{i}_R"] = np.zeros(len(nist17_pred))
    nist17_pred[f"S_{i}_B"] = np.zeros(len(nist17_pred))
    for col in cols:
        nist17_pred["d_" + col] = (nist17_pred.loc[:, col] -
                               nist17_pred.loc[:, "RI_X"]).abs()
        nist17_pred.loc[
            nist17_pred["d_" +
                    col] >= nist17_pred["d_" + col].quantile(float(
                        (100 - i) / 100)), f"S_{i}"] += 1

        nist17_pred["d_" + col +
                "_R"] = nist17_pred.loc[:, "d_" +
                                    col] / nist17_pred.loc[:, "p_median"].abs()
        nist17_pred.loc[nist17_pred["d_" + col + "_R"] >= nist17_pred["d_" + col + "_R"].
                    quantile(float((100 - i) / 100)), f"S_{i}_R"] += 1
        nist17_pred.loc[(nist17_pred["d_" + col + "_R"] >= nist17_pred["d_" + col + "_R"].
                    quantile(float((100 - i) / 100)))&(nist17_pred["d_" +
                    col] >= nist17_pred["d_" + col].quantile(float(
                        (100 - i) / 100))), f"S_{i}_B"] += 1


In [ ]:
nist17_pred[["d_" + x for x in ["RI_1D", "RI_MLP", "RI_CB", "RI_2D", "RI_SVR"]]].corr()

## STD Statistics PREDS

In [ ]:
nist17_pred[(nist17_pred["S_5"]==5)&(nist17_pred["S_5_R"]==5)].describe()["d_median"]

In [ ]:
nist17_pred[(nist17_pred["S_5_B"]==5)].describe()["d_median"]

In [ ]:
cols = ["RI_1D", "RI_MLP", "RI_CB", "RI_2D", "RI_SVR"]
fig, axs = plt.subplots(1, 5, figsize=(20, 3))
for i, col in enumerate(cols):
    cols_out = [x for x in cols if x != col]
    nist17_pred["p_std"] = nist17_pred.loc[:, cols_out].std(axis=1)
    nist17_pred[f"S_4"] = np.zeros(len(nist17_pred))
    nist17_pred[f"S_4_R"] = np.zeros(len(nist17_pred))
    for it in cols_out:
        nist17_pred.loc[nist17_pred["d_" + it] >= nist17_pred["d_" + it].
                    quantile(float((100 - 5) / 100)), "S_4"] += 1
        nist17_pred.loc[nist17_pred["d_" + it + "_R"] >= nist17_pred["d_" + it + "_R"].
                    quantile(float((100 - 5) / 100)), "S_4_R"] += 1
    std_mean = nist17_pred.groupby("S_4", as_index=False)["p_std"].mean()["p_std"]
    std_median = nist17_pred.groupby("S_4",
                                 as_index=False)["p_std"].median()["p_std"]

    axs[i].plot(np.arange(5), std_mean, color="b", label="mean")
    axs[i].plot(np.arange(5), std_median, color="g", label="median")
    axs[i].text(2, 40, f"Without {str(col).lstrip('RI_')}")
    axs[i].legend()
    axs[i].grid()
fig.suptitle('STD of predictions by number of "bad" absolute marks')

In [ ]:
cols = ["RI_1D", "RI_MLP", "RI_CB", "RI_2D", "RI_SVR"]
fig, axs = plt.subplots(1, 5, figsize=(20, 3))
for i, col in enumerate(cols):
    cols_out = [x for x in cols if x != col]
    nist17_pred["p_std"] = nist17_pred.loc[:, cols_out].std(axis=1)
    nist17_pred[f"S_4"] = np.zeros(len(nist17_pred))
    nist17_pred[f"S_4_R"] = np.zeros(len(nist17_pred))
    for it in cols_out:
        nist17_pred.loc[nist17_pred["d_" + it] >= nist17_pred["d_" + it].
                    quantile(float((100 - 5) / 100)), "S_4"] += 1
        nist17_pred.loc[nist17_pred["d_" + it + "_R"] >= nist17_pred["d_" + it + "_R"].
                    quantile(float((100 - 5) / 100)), "S_4_R"] += 1
    std_mean = nist17_pred.groupby("S_4_R", as_index=False)["p_std"].mean()["p_std"]
    std_median = nist17_pred.groupby("S_4_R",
                                 as_index=False)["p_std"].median()["p_std"]

    axs[i].plot(np.arange(5), std_mean, color="b", label="mean")
    axs[i].plot(np.arange(5), std_median, color="g", label="median")
    axs[i].text(2, 40, f"Without {str(col).lstrip('RI_')}")
    axs[i].legend()
    axs[i].grid()
fig.suptitle('STD of predictions by number of "bad" relative marks')

In [ ]:
cols = ["RI_1D", "RI_MLP", "RI_CB", "RI_2D", "RI_SVR"]
nist17_pred["p_std"] = nist17_pred.loc[:, cols].std(axis=1)

FIG: Number of entries by number of bad marks

In [ ]:
num_s = nist17_pred.groupby("S_5_B",as_index=False)["Formula"].count()["Formula"]
fig,ax = plt.subplots(1,1,figsize=(4,3))
ax.bar(np.arange(1,6),num_s[1:])
ax.set_xlabel('Number of "bad" marks')
ax.set_ylabel('# of entries')
fig.suptitle('# of entries by number of "bad" marks')
plt.grid(axis="y")

In [ ]:
num_s = nist17_pred.groupby("S_5_B",as_index=False)["Formula"].count()["Formula"]/len(nist17_pred)*100
print(num_s)
fig,ax = plt.subplots(1,1,figsize=(4,3))
ax.bar(np.arange(1,6),num_s[1:])
ax.set_xlabel('Number of "bad" marks')
ax.set_ylabel('% of entries')
fig.suptitle('% of entries by number of "bad" marks')
plt.grid(axis="y")

FIG: STD predictions by number of bad marks

In [ ]:
std_mean = nist17_pred.groupby("S_5_B",as_index=False)["p_std"].mean()["p_std"]
std_median = nist17_pred.groupby("S_5_B",as_index=False)["p_std"].median()["p_std"]
fig,ax = plt.subplots(1,1,figsize=(4,3),dpi=1000)
ax.plot(np.arange(6),std_mean, color = "b", label="mean")
ax.plot(np.arange(6),std_median, color = "g", label="median")
ax.set_xlabel('Number of "yellow cards"', fontdict={"fontsize":12})
ax.set_ylabel('Standard deviation, i.u.',fontdict={"fontsize":12})
# fig.suptitle('STD of predictions by number of "yellow cards"')
plt.legend(fontsize=12)
plt.grid()
# plt.savefig("../Data/OUT/fig_std.jpg",bbox_inches="tight")
# plt.savefig("../Data/OUT/fig_std.eps",bbox_inches="tight")

In [ ]:
nist17_pred.groupby("S_5",as_index=False).describe(percentiles=[0.5,0.75,0.9,0.95])["p_std"]

Probabaility density distributions

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(4, 3))
for i in range(6):
    ax.hist(nist17_pred[nist17_pred["S_5"] == i]["p_std"],
            bins=250,
            density=True,
            histtype="step",
            label=f"{i} marks")
ax.set_ylabel('Probability density')
ax.set_xlabel('Standard deviation, i.u.')
ax.set_xbound(0,200)
fig.suptitle('Probability density for STD in 1 and 5 marks groups')
plt.legend()

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(4, 3))
for i in range(6):
    ax.hist(nist17_pred[nist17_pred["S_5"] == i]["p_std_r"],
            bins=250,
            density=True,
            histtype="step",
            label=f"{i} marks")
ax.set_ylabel('Probability density')
ax.set_xlabel('Standard deviation / RI value')
ax.set_xbound(0,0.2)
fig.suptitle('Probability density for STD in 1 and 5 marks groups')
plt.legend()

FIG: probability denisities

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(4,3),dpi=1000)
plt.hist(nist17_pred[nist17_pred["S_5_B"] == 5]["p_std"],
         bins=100,
         density=True,
         color="red",
         histtype="step",
         label="5 marks")
plt.hist(nist17_pred[nist17_pred["S_5_B"] == 1]["p_std"],
         bins=100,
         density=True,
         color="blue",
         histtype="step",
         label="1 mark")
plt.hist(nist17_pred[nist17_pred["S_5_B"] == 0]["p_std"],
         bins=20,
         density=True,
         color="green",
         histtype="step",
         label="0 marks")
ax.set_ylabel('Probability density',fontsize=12)
ax.set_xlabel('Standard deviation, i.u.',fontsize=12)
ax.set_xbound(0,400)
# fig.suptitle('Probability density for STD in 1 and 5 marks groups')
plt.legend(fontsize=12)
# fig.savefig("../Data/OUT/fig_denst.jpg",bbox_inches="tight")
# fig.savefig("../Data/OUT/fig_denst.eps",bbox_inches="tight")

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(4,3))
plt.hist(nist17_pred[nist17_pred["S_5"] == 5]["p_std_r"],
         bins=200,
         density=True,
         color="b",
         histtype="step",
         label="5 marks")
plt.hist(nist17_pred[nist17_pred["S_5"] == 1]["p_std_r"],
         bins=200,
         density=True,
         color="g",
         histtype="step",
         label="1 mark")
plt.hist(nist17_pred[nist17_pred["S_5"] == 0]["p_std_r"],
         bins=100,
         density=True,
         color="r",
         histtype="step",
         label="0 marks")
ax.set_ylabel('Probability density')
ax.set_xlabel('Standard deviation / RI value')
ax.set_xbound(0,0.2)
fig.suptitle('Probability density for STD in 1 and 5 marks groups')
plt.legend()

## Comparing NIST versions

In [ ]:
from rdkit.Chem.MolStandardize import rdMolStandardize
uncharger = rdMolStandardize.Uncharger()

def get_non_isomeric(smiles):
    mol = Chem.MolFromSmiles(smiles)
    if mol:
        return Chem.MolToSmiles(mol,isomericSmiles=False)
    else:
        return None
def get_molfs(smiles):
    mol = Chem.MolFromSmiles(smiles)
    if mol:
        return CalcMolFormula(mol)
    else:
        return None

In [ ]:
def compare_by_smiles(nist_db, pred_db):
    nist_err_idx = set()
    preds_nf_idx = []
    nist_mp_idx = set()
    preds_mp_idx = []
    preds_mp_phases =[]
    for i, val in tqdm(pred_db.iterrows(), total=len(pred_db)):
        filtered = nist_db[(nist_db["SMILES_NONISO"] == val["SMILES_NONISO"])
                           & (np.abs(nist_db["RI"] - val["RI_X"]) < 0.1)]
        if len(filtered) == 1:
            nist_err_idx.add(filtered.index.tolist()[0])
        elif len(filtered) == 0:
            preds_nf_idx.append(i)
        elif len(filtered) > 1:
            err_filtered = pred_db[(pred_db["SMILES_NONISO"] == val["SMILES_NONISO"])
                           & (np.abs(pred_db["RI_X"] - val["RI_X"]) < 0.1)]
            if len(err_filtered)==len(filtered):
                nist_err_idx.update(filtered.index.tolist())
            else:
                preds_mp_idx.append(i)
                preds_mp_phases.extend([val["ColType"]]*len(filtered))
                nist_mp_idx.update(filtered.index.tolist())
    return {
        "nist_err_idx": nist_err_idx,
        "preds_nf_idx": preds_nf_idx,
        "preds_mp_idx": preds_mp_idx,
        "preds_mp_phases":preds_mp_phases,
        "nist_mp_idx": nist_mp_idx
    }


In [ ]:
nist17_err = nist17_pred[(nist17_pred["S_5"]==5)&(nist17_pred["S_5_R"]==5)].copy()
nist17_err["SMILES_NONISO"] = list(map(get_non_isomeric,nist17_err["Formula"].tolist()))
nist17_err["EL_FORMULA"] = list(map(get_molfs,nist17_err["Formula"].tolist()))
nist17_err

Loading 2020 NIST

In [ ]:
compounds_20 = []
with open("../Data/nist20_RI.txt","r") as f:
    headers = f.readline().rstrip("\n").split("|")
    print(headers)
    for line in tqdm(f):
        compounds_20.append(line.rstrip("\n").split("|"))


In [ ]:
nist20 = pd.DataFrame(compounds_20,columns=headers)[["CAN_SMILES","CAN_NAME","EL_FORMULA","RI","RI_TYPE","SOURCE",'COLUMN_TYPE', 'STAT_PH_TYPE', 'ST_PH']]
nist20 = nist20[nist20["STAT_PH_TYPE"]!="Standard_polar"].copy()
nist20["SMILES_NONISO"] = list(map(get_non_isomeric,tqdm(nist20["CAN_SMILES"].tolist())))
nist20["RI"] = nist20["RI"].astype(float)
nist20

Comparing NIST RI 17 ERR vs NIST RI 20

In [ ]:
def find_changed(nist20_db, nist17_err_db):
    nist17_del_idx = set()
    nist17_no_ch_idx = set()
    nist20_no_ch_idx = set()
    nist20_added_idx = set()
    nist17_merge_idx = set()
    nist20_merge_idx = set()
    nist20_ch_idx = set()
    nist17_ch_idx = set()
    nist20_ch_ri_idx = set()
    nist17_ch_ri_idx = set()
    nist17_ch = []
    nist17_idx = set()
    nist20_db = nist20_db.loc[:,["SMILES_NONISO","RI","ST_PH","SOURCE"]]
    nist17_err_db = nist17_err_db.loc[:,["SMILES_NONISO","RI","ST_PH","SOURCE"]]
    for i, val in tqdm(nist17_err_db.iterrows(), total=len(nist17_err_db)):
        nist20_f_smi = nist20_db.loc[(
            nist20_db["SMILES_NONISO"] == val["SMILES_NONISO"]),]
        
        if len(nist20_f_smi) == 0:
            nist17_del_idx.add(i)
        else:
            eqs20 = nist20_f_smi[nist20_f_smi.eq(val,axis=1).all(axis=1)]
            eqs17 = nist17_err_db[nist17_err_db.eq(val,axis=1).all(axis=1)]
            if len(eqs20)==0:
                eqs20_ri = nist20_f_smi[nist20_f_smi.eq(val,axis=1)==[True,False,True,True]]
                if len(eqs17)<=len(eqs20_ri):
                    nist20_ch_ri_idx.update(eqs20_ri.index.tolist())
                    nist17_ch_ri_idx.update(eqs17.index.tolist())
                else:
                    nist20_ch_idx.update(eqs20_ri.index.tolist())
                    nist17_ch_idx.update(eqs17.index.tolist())
            elif len(eqs17)==len(eqs20):
                nist17_no_ch_idx.update(eqs17.index.tolist())
                nist20_no_ch_idx.update(eqs20.index.tolist())
            elif len(eqs17)>len(eqs20):
                nist17_merge_idx.update(eqs17.index.tolist())
                nist20_merge_idx.update(eqs20.index.tolist())
            elif len(eqs17)<len(eqs20):
                print("addition")                
            # nist17_idx.update(eqs17.index.tolist())
            # if len(eqs17)>1:
            #     # print("multiple")
            #     print(eqs17)

            # nist20_f_smi_ri = nist20_f_smi[np.abs(nist20_f_smi["RI"] - val["RI"]) < 0.1]
            # nist17_f_smi_ri = nist17_f_smi[(np.abs(nist17_f_smi["RI"] - val["RI"]) < 0.1)]
            # if set(nist17_f_smi_ri["SOURCE"].tolist())==set(nist20_f_smi_ri["SOURCE"].tolist()):
            #     if set(nist17_f_smi_ri["ST_PH"].tolist())==set(nist20_f_smi_ri["ST_PH"].tolist()):
            #         if len(nist20_f_smi_ri)==len(nist17_f_smi_ri):
            #             nist17_no_ch_idx.update(nist17_f_smi_ri.index.tolist())
            #             nist20_no_ch_idx.update(nist20_f_smi_ri.index.tolist())
            # else:
            #     nist17_ch_idx.update(nist17_f_smi_ri.index.tolist())
            #     nist20_ch_idx.update(nist20_f_smi_ri.index.tolist())
    return {
        "nist17_del_idx": nist17_del_idx,
        "nist20_no_ch_idx": nist20_no_ch_idx,
        "nist17_no_ch_idx": nist17_no_ch_idx,
        "nist20_ch_idx": nist20_ch_idx,
        "nist17_ch_idx": nist17_ch_idx,
        "nist20_ch_ri_idx": nist20_ch_ri_idx,
        "nist17_ch_ri_idx": nist17_ch_ri_idx,
        # "nist17_idx": nist17_idx,
        "nist17_merge_idx":nist17_merge_idx,
        "nist20_merge_idx":nist20_merge_idx,
        "nist17_ch": nist17_ch
    }


In [ ]:
ch_dict = find_changed(nist20,nist17_err)
for key,val in ch_dict.items():
    print(key,len(val))

In [ ]:
nist17_pred.loc[ch_dict["nist17_pred_merge_idx"]].groupby(["CAN_SMILES","RI","ST_PH","SOURCE"],as_index=False).mean()

In [ ]:
nist20.loc[ch_dict["nist20_merge_idx"]].sort_values("CAN_SMILES")

In [ ]:
nist17_err.loc[ch_dict["nist17_no_ch_idx"]]

In [ ]:
nist17_err_del = nist17_err.loc[ch_dict["nist17_del_idx"]].groupby("SOURCE")
nist17_err_del = dict(zip(nist17_err_del.size().sort_values(ascending=False).index.tolist(),nist17_err_del.size().sort_values(ascending=False).tolist()))

In [ ]:
nist17_err_ch = nist17_err.loc[ch_dict["nist17_ch_ri_idx"]].groupby("SOURCE")
nist17_err_ch = dict(zip(nist17_err_ch.size().sort_values(ascending=False).index.tolist(),nist17_err_ch.size().sort_values(ascending=False).tolist()))

Looking up the largest contributors / submissions

In [ ]:
nist17_cbtrs = nist17.groupby("SOURCE").size().sort_values(ascending=False)
nist17_cbtrs = dict(zip(nist17_cbtrs.index.tolist(),nist17_cbtrs.tolist()))

50 biggest contributors sorted by their potential error percentage

In [ ]:
nist17_err_gb = nist17_err.groupby("SOURCE")
cbtr_df = pd.DataFrame(columns = ["SOURCE","ERR_CNT","DEL_CNT","CH_CNT","ALL_CNT"])
sources = nist17_err.groupby("SOURCE").size().sort_values(ascending=False).index.tolist()
# sources = nist17.groupby("SOURCE").size().sort_values(ascending=False).index.tolist()
cbtr_np = np.zeros((len(sources),4))
cbtr_np[:,0] = nist17_err_gb.size().sort_values(ascending=False).tolist()
for i,val in enumerate(sources):
    cbtr_np[i,1] = nist17_err_del.get(val,0)
    cbtr_np[i,2] = nist17_err_ch.get(val,0)
    cbtr_np[i,3] = nist17_cbtrs.get(val,0)
cbtr_df["SOURCE"] = sources
cbtr_df[["ERR_CNT","DEL_CNT","CH_CNT","ALL_CNT"]] = cbtr_np
cbtr_df["ERR_PERC"] = cbtr_df["ERR_CNT"] / cbtr_df["ALL_CNT"]*100
cbtr_df = cbtr_df.sort_values("ALL_CNT", ascending=False)[:50]
cbtr_df = cbtr_df.sort_values("ERR_PERC", ascending=True)
cbtr_df

Looking up Zaikin contributions

In [ ]:
cbtr_df[[True if "ZAI" in x else False for x in cbtr_df["SOURCE"]]].sum()

In [ ]:
nist17[nist17["SOURCE"]=="1999SOK99-104"]

In [ ]:
fig,ax = plt.subplots(1,1, figsize=(15,5))
plt.xticks(rotation=90)
# plt.tight_layout()
xs = cbtr_df["SOURCE"]
ys = cbtr_df["ERR_PERC"]
ax.bar(xs,ys, color ="blue", label = "Errors in NIST 2017")
mask = [True if "ZAI" in x else False for x in xs]
ax.bar(xs[mask],ys[mask], color ="teal")
ax.hlines(y=1,xmin=0,xmax=len(ys), color="red")
ax.margins(x=0.01)
ax.set_xlabel("Contributors/Submissions", fontsize=16)
ax.set_ylabel("Percentage of erroneous entries", fontsize=16)
ax.text(0,1.1,"1% of errors per submission")
fig.suptitle("Percentage of erroneous entries by largest NIST RI submissions",fontsize=20)

FIG: Contributors

In [ ]:
fig,ax = plt.subplots(1,1, figsize=(7.5,4),dpi=1000)
plt.xticks(rotation=90)
# plt.tight_layout()
xs = cbtr_df["SOURCE"][-30:]
ys = cbtr_df["ERR_PERC"][-30:]
ys_ch = (cbtr_df["CH_CNT"]/cbtr_df["ALL_CNT"]*100)[-30:]
ys_ch_del = ((cbtr_df["CH_CNT"]+cbtr_df["DEL_CNT"])/cbtr_df["ALL_CNT"]*100)[-30:]
ax.bar(xs,ys, color ="blue", label = "Errors in NIST 2017")
ax.bar(xs,ys_ch_del, color ="purple",label = "Deleted in NIST 2020")
ax.bar(xs,ys_ch, color ="green",label = "Corrected in NIST 2020")
# ax.bar(xs[mask],ys[mask], color ="green")
# ax.hlines(y=ys[np.invert(mask)].mean(),xmin=0,xmax=len(ys), color="red")
ax.margins(x=0.01)
[t.set_color('teal') for t in ax.xaxis.get_ticklabels() if "zai" in str(t).lower()]
[t.set_color('maroon') for t in ax.xaxis.get_ticklabels() if "tod" in str(t).lower()]
ax.set_xlabel("Contributors/Submissions", fontsize=12)
ax.set_ylabel("Percentage of erroneous entries", fontsize=12)
# fig.suptitle("Percentage of erroneous entries by largest NIST RI submissions",fontsize=20)
plt.legend(fontsize=12)
# fig.savefig("../Data/OUT/fig_cbtrs.jpg",bbox_inches="tight")
# fig.savefig("../Data/OUT/fig_cbtrs.eps",bbox_inches="tight")

NIST RI 20 entries with changed RI

In [ ]:
nist20.loc[ch_dict["nist20_ch_ri_idx"]].groupby(["SMILES_NONISO","ST_PH","SOURCE"],as_index=False).size().sort_values("size",ascending=False)

NIST RI 17 entries with changed RI

In [ ]:
nist17.loc[ch_dict["nist17_ch_ri_idx"]].groupby(["SMILES_NONISO","ST_PH","SOURCE"],as_index=False).size().sort_values("size",ascending=False)

In [ ]:
nist17_smi_changed = nist17.loc[nist17_ch_dict["nist_ch_idx"],:]
nist17_smi_changed